In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Feb 23 06:55:11 2025

@author: josgomezru modified from dariograna

Reference: Grana and de Figueiredo, 2021, SeReMpy - Equations 5 and 6
           Li and Liu, 2019, Equations 6 and 7
"""

# Geostatistical Petrophysical inversion Driver %%
# In this script we apply the Ensenmble petrophysical inversion method
# (Liu and Grana, 2018) to predict the petrophysical properties 
# from seismic data.

'\nCreated on Sun Feb 23 06:55:11 2025\n\n@author: josgomezru modified from dariograna\n\nReference: Grana and de Figueiredo, 2021, SeReMpy - Equations 5 and 6\n           Li and Liu, 2019, Equations 6 and 7\n'

In [14]:
#Load library
import sys
sys.path.append('/workspaces/SeReMpy/Applications')
sys.path.append('/workspaces/SeReMpy/Data')

In [15]:
from scipy.io import loadmat
import matplotlib.pyplot as plt
from scipy import signal
import numpy as np
from numpy import matlib

from context import SeReMpy
from SeReMpy.Inversion import *

In [16]:
#% Application4
# Load data (seismic data and time)
x = np.loadtxt('/workspaces/SeReMpy/Data/1Ddatalog.dat')
Rho = x[:,4].reshape(-1, 1)
Time = x[:,6].reshape(-1, 1)
Vp = x[:,7].reshape(-1, 1)
Vs = x[:,8].reshape(-1, 1)
s= np.loadtxt('/workspaces/SeReMpy/Data/1Ddataseis.dat')
Sfar = s[:,0].reshape(-1, 1)
Smid = s[:,1].reshape(-1, 1)
Snear = s[:,2].reshape(-1, 1)
TimeSeis = s[:,3].reshape(-1, 1)

In [21]:
#% Initial parameters
#Vs_bar_omega means average Vs across the interface.
#delta_rho is the difference of rho across the interface.
#gamma = Vs_bar_omega / Vp_bar_omega
#rRho = delta_rho / rho_bar
#rXip = delta_xip / xip_bar
#rXis = delta_xis / xis_bar

In [22]:
def c_theta(gamma, theta):
    return 0.5 * (1 - 4 * gamma**2 * np.sin(theta)**2)

def d_theta_omega(theta, omega):
    return -0.5 * np.pi * (1 + np.tan(theta)**2) * np.log(omega_r / omega)

def e_theta_omega(theta, omega, gamma):
    return 4 * gamma / np.pi * np.sin(theta)**2 * np.log(omega_r / omega)

def R(theta, omega, gamma):
    return c_theta(gamma, theta) * rRho + d_theta_omega(theta, omega) * rXip + e_theta_omega(theta, omega, gamma) * rXis